In [ ]:
import sys
from ctypes import CDLL
# This will crash a Linux or Mac system
# equivalent calls can be made on Windows
# Uncomment these lines if you would like to see the segfault
# dll = 'dylib' if sys.platform == 'darwin' else 'so.6' # libc = CDLL("libc.%s" % dll)
# libc.time(-1) # BOOM!!

sys.stdout and sys.stderr

In [ ]:
print("hi, stdout")

In [ ]:
from __future__ import print_function 
print('hi, stderr', file=sys.stderr)

All output is displayed asynchronously as it is generated in the Kernel. If you execute the next cell, you will see the output one piece at a time, not all at the end.

In [ ]:
import time, sys 
for i in range(8):
    print(i)
    time.sleep(0.5)
    

Large outputs
To better handle large outputs, the output area can be collapsed. Run the following cell and then single- or double- click on the active area to the left of the output:

In [ ]:
for i in range(50): 
    print(i)

Beyond a certain point, output will scroll automatically:

In [ ]:
for i in range(500): 
    print(2**i - 1)

Embedded code 
embed code meant for illustration instead of execution in Python:


In [ ]:
def f(x):
    """a docstring"""
    return x**2

LaTeX equations
Courtesy of MathJax, you can include mathematical expressions both inline: 𝑒𝑖𝜋 + 1 = 0 and displayed:


In [ ]:
#Inline expressions can be added by surrounding the latex code with $:
$e^{i\pi} + 1 = 0$

In [ ]:
#Expressions on their own line are surrounded by \begin{equation} and \end{equation}:
\begin{equation} 
e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i 
\end{equation}


GitHub flavored markdown
The Notebook webapp supports Github flavored markdown meaning that you can use triple backticks for code blocks:

In [ ]:
'''python
print "Hello World" 
'''
#you can run python and javascript using the three (''') at the beging of the code and at the end 
 
'''
console.log("Hello World")
'''

Local files
If you have local files in your Notebook directory, you can refer to these files in Markdown cells directly:

In [ ]:
[subdirectory/] <filename>

#For example, in the images folder, we have the Python logo:

<img src="../images/python_logo.svg" />

#and a video with the HTML5 video tag:
    

<video controls src="../images/animation.m4v">animation</video>

Files are stored in cell metadata and will be automatically scrubbed at save-time if not referenced. You can recognized attached images from other files by their url that starts with attachment:. For the image above:

In [ ]:
![pycon-logo.jpg](attachment:pycon-logo.jpg)
#Keep in mind that attached files will increase the size of your notebook.
#You can manually edit the attachment by using the View > Cell Toolbar > Attachment menu, but you should not need to.

Importing Jupyter Notebooks as Modules

Import hooks typically take the form of two objects:
1. a Module Loader, which takes a module name (e.g. 'IPython.display'), and returns a Module
2. a Module Finder, which figures out whether a module might exist, and tells Python what Loader to use


In [ ]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    
    
    """find a notebook, given its fully qualified name and an optional path
This turns "foo.bar" into "foo/bar.ipynb"
and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar does not exist.
"""

    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = [''] 
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb") 
        if os.path.isfile(nb_path):
            return nb_path
# let import Notebook_Name find "Notebook Name.ipynb" nb_path = nb_path.replace("_", " ")
    if os.path.isfile(nb_path):
        return nb_path



Notebook Loader
Here we have our Notebook Loader. It’s actually quite simple - once we figure out the filename of the module, all it does is:

1. load the notebook document into memory 2. create an empty Module
3. execute every cell in the Module namespace

Since IPython cells can have extended syntax, the IPython transform is applied to turn each of these cells into their pure-Python counterparts before executing them. If all of your notebook cells are pure-Python, this step is unnecessary.

In [ ]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks""" 
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
def load_module(self, fullname):
    """import a notebook as a module"""
    path = find_notebook(fullname, self.path)
    
    print ("importing Jupyter notebook from %s" % path)
    
    # load the notebook object
    with io.open(path, 'r', encoding='utf-8') as f:
        nb = read(f, 4)
    # create the module and add it to sys.modules # if name in sys.modules:
# return sys.modules[name]
    mod = types.ModuleType(fullname)
    mod.__file__ = path
    mod.__loader__ = self 
    mod.__dict__['get_ipython'] = get_ipython 
    sys.modules[fullname] = mod
# extra work to ensure that magics that would affect the user_ns 
# actually affect the notebook module's ns
    save_user_ns = self.shell.user_ns
    self.shell.user_ns = mod.__dict__
    try:
        for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source) 
                # run the code in themodule
                exec(code, mod.__dict__)
    finally:
        self.shell.user_ns = save_user_ns 
        return mod 